# Dependencies and Media Pipe Setup

In [1]:
!pip install mediapipe opencv-python
!pip install imageio
# !pip install pafy
# Outdated, do this into project source instead: https://stackoverflow.com/questions/57504520/editing-a-python-package
# Change line about Youtube dislikes in pafy/backend_youtube_dl

You should consider upgrading via the 'c:\users\notal\appdata\local\programs\python\python39\python.exe -m pip install --upgrade pip' command.


You should consider upgrading via the 'c:\users\notal\appdata\local\programs\python\python39\python.exe -m pip install --upgrade pip' command.


In [2]:
import mediapipe as mp
import cv2
import numpy as np
import uuid
import os
import imageio
import math   
import pafy

In [3]:
mp_drawing = mp.solutions.drawing_utils
mp_hands = mp.solutions.hands

# Using Camera, Detecting Hands

In [3]:
# Video Capture
cap = cv2.VideoCapture(0)

# Instantiation
# Detection: Initial detection, tracking: after initial detection
# Put max_num_hands to render more than two hands
with mp_hands.Hands(min_detection_confidence=0.8, min_tracking_confidence=0.5) as hands:
    while cap.isOpened():
        # Rendering capture
        ret, frame = cap.read()
        
        # Change the color to work with MediaPipe
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        
        # Flip horizontal
        image = cv2.flip(image, 1)
        
        image.flags.writeable = False
        
        # Detection
        results = hands.process(image)
        image.flags.writeable = True
        
        # Change color back to default
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        
        print(results)
        
        # Draw onto detection
        # Pass drawing specs to change color of skeleton
        if results.multi_hand_landmarks:
            for num, hand in enumerate(results.multi_hand_landmarks):
                mp_drawing.draw_landmarks(image, hand, mp_hands.HAND_CONNECTIONS)
        
        
        cv2.imshow("Hand Tracking", image)

        # q is exit key
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

cap.release()
cv2.destroyAllWindows()

<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.soluti

# Overlay function

In [4]:
def overlay(name, image, filter_img, hand_landmarks, display=True):
#     image:          Image to put filter on
#     filter_img:     The filter
#     hand_landmarks: Landmarks of hand
#     display:        Bool to diplay image

    image_copy = image.copy()
    
    image_height, image_width, _ = image.shape
    
    landmarks = []
    for points in hand_landmarks.landmark:
            landmarks.append([int(points.x * image_width),
                            int(points.y * image_height)])
    
    _, _, hand_width, hand_height = cv2.boundingRect(np.array(landmarks))
    
    landmarks = np.array(landmarks)
    
    try:

        filter_img_height, filter_img_width, _ = filter_img.shape
        
#         aspectRatio = math.ceil(int(filter_img_width/filter_img_height))
        
        aspectRatio = filter_img_width/filter_img_height
        
        resized_filter_img = cv2.resize(filter_img, (int(aspectRatio*hand_height),
                                                     hand_height))
        
        
        filter_img_height, filter_img_width, _ = resized_filter_img.shape

        _, filter_img_mask = cv2.threshold(cv2.cvtColor(resized_filter_img, cv2.COLOR_BGR2GRAY),
                                                25, 255, cv2.THRESH_BINARY_INV)
          
         
        center = landmarks.mean(axis=0).astype("int")
         
                
        location = (int(center[0]-filter_img_width/2), int(center[1]-filter_img_height/2))
          

        ROI = image[location[1]: location[1] + filter_img_height,
                       location[0]: location[0] + filter_img_width]
        
        
        resultant_image = cv2.bitwise_and(ROI, ROI, mask=filter_img_mask)
        resultant_image = cv2.add(resultant_image, resized_filter_img)

        image_copy[location[1] : location[1] + filter_img_height,
                  location [0] : location[0] + filter_img_width] = resultant_image
    except Exception as e:
        print(e)
        pass

    if display:
        return
    else:
        return image_copy
    

# Hand Filter Test Code

In [5]:
racketFilter = cv2.imread('Filters/pingPongRacket.png')

# fireFilter = imageio.get_reader('Filters/firePlume.gif')
# fireFilter = cv2.VideoCapture('Filters/firePlume.gif')

url = "https://www.youtube.com/watch?v=aKn0HddzuWM"
videoPafy = pafy.new(url)
best = videoPafy.getbest(preftype="mp4")
fireFilter = cv2.VideoCapture(best.url)

fireFilterCounter = 0


# Video Capture
cap = cv2.VideoCapture(0)

# Put max_num_hands to render more than two hands
with mp_hands.Hands(min_detection_confidence=0.8, min_tracking_confidence=0.5) as hands:
    while cap.isOpened():
        # Rendering capture
        ret, frame = cap.read()
        
        # Change the color to work with MediaPipe
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        
        # Flip horizontal
        image = cv2.flip(image, 1)
        
        image.flags.writeable = False
        
        # Detection
        results = hands.process(image)
        image.flags.writeable = True
        
        # Change color back to default
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        
        print(results)
        
#         fireFilterFrame = fireFilter.get_data(fireFilterCounter)
        _, fireFilterFrame = fireFilter.read()
        
        fireFilterCounter += 1
        
#         if fireFilterCounter == fireFilter.get_length():
#             fireFilterCounter = 0

        if fireFilterCounter == fireFilter.get(cv2.CAP_PROP_FRAME_COUNT):
            fireFilter.set(cv2.CAP_PROP_POS_FRAMES, 0)
            fireFilterCounter = 0
       
        if results.multi_hand_landmarks:
            for num, hand in enumerate(results.multi_hand_landmarks):
#                 if num == 0:
#                     image = overlay('racketFilter', image, racketFilter, hand, display=False)
                if num == 0:
                    image = overlay('fireFilter', image, fireFilterFrame, hand, display=False)
        
        
        cv2.imshow("Hand Tracking", image)

        # q is exit key
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

cap.release()
cv2.destroyAllWindows()

<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.soluti

<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.soluti